<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Notebook</p></tp>
            <tp><p style="font-size:150%;text-align:center">Gramáticas Independientes del Contexto</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


## Objetivo <a class="anchor" id="inicio"></a>

En este notebook nos familiarizaremos con el uso de la librería `nltk` para procesar Gramáticas Independientes del Contexto. Veremos cómo crear gramáticas, generar cadenas y encontrar los árboles de análisis. También escribiremos nuestra propia "toy grammar" para el español.

Este notebook está basado en los capítulos 8 y 9 de [1], el cual puede consultarse en la [página web](https://www.nltk.org/book/ch08.html).  

[Ir a ejercicio 1](#ej1)

## Dependencias

Al iniciar el notebook o reiniciar el kerner, se pueden cargar todas las dependencias de este notebook al correr las siguientes celdas. Este también es el lugar para instalar las dependencias que podrían hacer falta.

In [ ]:
# En linux o mac
#!pip3 install nltk
#!pip3 install svgling
#!pip3 install pandas
#!pip3 install random

# En windows
#!py -m pip install nltk
#!py -m pip install svgling
#!py -m pip install pandas
#!py -m pip install random


In [ ]:
import nltk
from nltk import CFG, parse
from nltk.grammar import FeatureGrammar
from nltk.tree import Tree
from nltk.parse import RecursiveDescentParser, FeatureEarleyChartParser
from nltk.parse.generate import generate
from random import sample
import pandas as pd

In [ ]:
import utils

## Secciones

Desarrollaremos la explicación en las siguientes secciones:

* [Una gramática muy simple.](#gram1)
* [Parsing.](#parsing)
* [Características gramaticales.](#caracs)
* [Evaluación de una gramática](#eval)


# Una gramática muy simple
<a class="anchor" id="gram1"></a>

([Volver al inicio](#inicio))

**Gramática abstracta**

Vamos a comenzar con el ejemplo que vimos en las diapositivas de una gramática muy sencilla:

$$
    \begin{split}
      A &\to 0A1\\
      A &\to B\\
      B &\to 2
    \end{split}
$$

Esta gramática podemos escribirla como una cadena e importarla mediante el método `fromstring()` de la clase `CFG`:

In [ ]:
reglas1 = """
A -> '0' A '1' | B
B -> '2'
"""
gram_abs = CFG.fromstring(reglas1)

Podemos ver que el objeto creado tiene el símbolo incial, que se puede visualizar mediante `start()`, y también las reglas de producción, mediante `productions()`:

In [ ]:
gram_abs.start()

In [ ]:
gram_abs.productions()

Observe que también hemos importado la función `generate`, mediante la cual podemos generar todas las cadenas que se puedan obtener usando la gramática. Como esta es una gramática recursiva (es decir, que al aplicar la regla de reescritura sobre una cadena podemos aplicarla de nuevo una y otra vez), debemos dar un límite a la profundidad de los árboles que queremos generar. En este caso, daremos la profundidad máxima de 5:

In [ ]:
list(generate(grammar=gram_abs, depth=5))

Observe que hemos generado tres cadenas, a saber, `00211`, `021` y `2`.

**Gramática sencilla para el español**

Veamos ahora cómo crear nuestra primera gramática para el español. Este primer ejemplo será muy sencillo, pues solo uniremos un término con un predicado para formar oraciones:

In [ ]:
reglas2 = """
O -> SN V
SN -> D N
V -> 'camina'
D -> 'un' | 'una'
N -> 'hombre' | 'mujer'
"""
gramatica = nltk.CFG.fromstring(reglas2)

Generamos ahora todas las oraciones que se puedan obtener mediante esta pequeña gramática. Como ell no es recursiva, no es necesario poner un límite de profundidad para los árboles generados.  

In [ ]:
list(generate(gramatica))

---

## Parsing <a class="anchor" id="parsing"></a>

([Volver al inicio](#inicio))

El parsing es un proceso mediante el cual se obtiene uno o todos los árboles de análisis de una cadena dada. Más adelante en el curso hablaremos con más calma sobre algunas maneras sencillas para hacer parsing. Por el momento, es importante saber que la librería `nltk` ya tiene implementada varias clases para realizar parsing.

**Gramática abstracta**

Veámos cómo se realiza el parsing mediante el algoritmo de descenso recursivo, usando como ejemplo la gramática abstracta definida más arriba:

In [ ]:
# Instanciamos un objeto que implementa 
# el método de descenso recursivo:
rd = RecursiveDescentParser(gram_abs, trace=2)
#                              ^
#                           gramática
#                           abstracta

# Creamos nuestra oración a analizar:
oracion1 = '0 2 1'.split()

# Realizamos el parsing:
trees = rd.parse(oracion1)

# Visualizamos (de manera lineal):
for t in trees:
    print(t)

Observe que el objeto tiene un atributo `trace`, el cual puede inicializarse con el valor 0 para evitar la visualización del proceso.

In [ ]:
# Instanciamos un objeto que implementa 
# el método de descenso recursivo:
rd = RecursiveDescentParser(gram_abs, trace=0)
#                                           ^
#                                        info a
#                                        visualizar

# Creamos nuestra oración a analizar:
oracion1 = '0 2 1'.split()

# Realizamos el parsing:
trees = rd.parse(oracion1)

# Visualizamos (de manera lineal):
for t in trees:
    print(t)

Observe también que la anterior visualización no es muy explícita para ver el árbol. En efecto, no se ve ningún árbol. Lo que obtenemos es una cadena con la representación lineal de uno. Para poder tener un árbol, podemos echar mano de la clase `Tree`:

In [ ]:
# Creamos un árbol usando el método fromstring:
tree = Tree.fromstring(str(t))

# Visualizamos el árbol:
tree

<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Visualice el árbol de análisis de cada una de las oraciones generadas mediante la "toy grammar" del español que definimos más arriba.

---

# Características gramaticales
<a class="anchor" id="caracs"></a>

([Volver al inicio](#inicio))

Hasta ahora, nuestra toy grammar genera oraciones no gramaticales, como "una hombre camina". Vamos a usar características semánticas para bloquear este tipo de combinaciones. Comencemos por implementar la característica del género, que se aplica tanto para sustantivos como para determinantes:

- GEN, género (m=masculino, f=femenino)

Observe que ahora vamos a usar una clase distinta, que se llama `FeatureGrammar`, mediante la cual implementamos la gramática con características. También tenemos que usar un parser distinto, que se llama `FeatureEarlyChartParser`: 

In [ ]:
# Definimos la gramática
toy_g = """
% start O
O -> SN V
SN[GEN=?g] -> D[GEN=?g] N[GEN=?g]
D[GEN=m] -> 'un'
D[GEN=f] -> 'una'
N[GEN=m] -> 'hombre'
N[GEN=f] -> 'mujer'
V -> 'camina'
"""

# Instanciamos el objeto
gramatica_f = FeatureGrammar.fromstring(toy_g)

# Instanciamos el parser
parser = FeatureEarleyChartParser(gramatica_f)

Vamos a probar el parser:

In [ ]:
tree1 = utils.parsear(o1, parser, verbose=True)
tree1

In [ ]:
tree2 = utils.parsear(o2, parser, verbose=True)
tree2

---

# Evaluación de una gramática

<a class="anchor" id="eval"></a>

([Volver al inicio](#inicio))

Es importante observar que una gramática nos permite clasificar cadenas como gramaticales (es decir, aquellas para las cuales el parser nos devolverá un árbol de análisis) o no gramaticales. Por ejemplo, la cadena "un hombre camina" es clasificada como gramatical por nuestra `toy_g` mientras que "un mujer camina" es clasificada como no gramatical. 

Adicionalmente, observe que la clasificación de una oración depende de una gramática. Por ejemplo, la gramática `reglas2` creada más arriba clasifica ambas cadenas como gramaticales. Veamos este resultado de manera gráfica mediante la siguiente tabla:

| | `reglas2` | `toy_g` | Gold |
| :---: | :---: | :---: | :---: |
| un hombre camina | gramatical | gramatical  | gramatical  |
| un mujer camina | gramatical | no gramatical | no gramatical |

El desempeño de las gramáticas lo evaluamos de acuerdo a un "gold standard". En este caso, el "gold" es un conjunto de oraciones, que nosotros consideramos independientemente como gramaticales.

¿Cómo evaluamos el desempeño de una gramática respecto al "gold"?

**Matriz de confusión**

Se definen las medidas de **precision**, **recall** y **accuracy** de acuerdo a la siguiente matriz de confusión:

<img src="./imagenes/confusion.png" width="500"/>

En otras palabras, la **precision** nos dice qué porcentaje es correcto de las oraciones que fueron clasificadas como gramaticales:

$$
\text{precision} = \frac{\text{true positives}}{\text{true positives} + \text{false positives}}
$$

El **recall** nos dice qué porcentaje de las oraciones gramaticales fue clasificado como gramatical:

$$
\text{recall} = \frac{\text{true positives}}{\text{true positives} + \text{false negatives}}
$$

El **accuracy** nos dice qué porcentaje de oraciones en todo el conjunto fue clasificado correctamente:

$$
\text{accuracy} = \frac{\text{true positives} + \text{true negatives}}{\text{true positives} + \text{false positives} + \text{true negatives} + \text{false negatives}}
$$

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

¿Cuál es el precision, recall y accuracy de las gramáticas `reglas2` y `toy_g`?

---

**Medida F1**

Una manera de combinar precision y recall en una sola medida se llama la medida F1, definida como:

$$
F1 = \frac{2*\textbf{precision}*\textbf{recall}}{\textbf{precision} + \textbf{recall}}
$$

La medida F1 toma valores entre 0 y 1. Cuanto más cercano a 1 significa un mejor desempeño.

Observe que F1(`reglas2`) = 0.666 y que F1(`toy_g`) = 1.

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))
    
Implemente la función F1.

In [ ]:
def F1(tp, fp, fn, tn, verbose=False):
    '''
    Devuelve la medida F1.
    Input:
        - tp, número de verdaderos positivos
        - fp, número de falsos positivos
        - fn, número de falsos negativos
        - tn, número de verdaderos negativos
        - verbose, booleano para presentar información
    '''
    pass
    # AQUÍ SU CÓDIGO
    
    # HASTA AQUÍ SU CÓDIGO

---

Observe la diferencia entre **accuracy** y F1. Considere el siguiente resultado (ficticio) de la clasificación dada por una gramática:

In [ ]:
data = pd.DataFrame({'oracion':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], 
                     'gold':['no']*9 + ['si'],
                     'resultado':['no']*10})
data

Esta gramática tiene accuracy = 0.909, pero un F1 = 0.

**Evaluando gramáticas**

Podemos usar la función `test_gramatica_carac()` para evaluar una gramática que hayamos escrito. Por ejemplo, podemos evaluar la sencilla gramática sin características `reglas2`, y comparar su desempeño con el de la gramática con características `toy_g`:

In [ ]:
print(reglas2)
f1 = utils.test_gramatica_carac('oraciones.csv', reglas2, trace=1)

In [ ]:
print(toy_g)
f1 = utils.test_gramatica_carac('oraciones.csv', toy_g, trace=1)

Vemos que el valor F1 de `toy_g` es 0.4, que es un poco mejor que el 0.3 de `reglas2`. No obstante, es posible hacerlo mejor.

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) 

Considere el dataset `combinaciones.csv`, el cual contiene los datos para el problema de clasificación. 

In [ ]:
data = pd.read_csv('combinaciones.csv')
mask = sample(range(data.shape[0]), 5)
data.iloc[mask]

Su tarea es escribir una gramática independiente del contexto con características gramaticales para obtener un valor F1 mayor a 0.95. Las características gramaticales que debe usar son las siguientes:

    - GEN, género (m=masculino, f=femenino)
        Para sustantivos, determinantes y sintagmas nominales.  
    - NUM, si es plural o no (sg=singular, pl=plural)
        Para sustantivos, determinantes, sintagmas nominales y verbos.

In [ ]:
gramatica = """
# AQUÍ SU CÓDIGO

# HASTA AQUÍ SU CÓDIGO
"""

In [ ]:
utils.test_gramatica_carac('oraciones.csv', gramatica, trace=1)

---

## En este notebook usted aprendió a

* Implementar gramáticas independientes del contexto usando las herramientas de `nltk`.
* Implementar características gramanticales.
* Usar un parser para obtener el árbol de análisis de una oración.
* Escribir gramáticas para clasificar oraciones como gramaticales o no.

## Bibliografía 

[1] Bird, S. and Klein, E. and Loper, E., 2009. Natural Language Processing with Python: Analyzing text with the Natural Language Toolkit. O’Reilly.